In [4]:
from praw import Reddit
from praw.models import Subreddit, Submission
import os
from dotenv import dotenv_values  
from dotenv import load_dotenv
from datetime import datetime, date, timedelta
import praw
import pandas as pd

In [12]:
config = dotenv_values(".env")

reddit = praw.Reddit(
client_id=config["CLIENT_ID"],
client_secret=config["CLIENT_SECRET"],
redirect_uri=config["REDIRECT_URL"],
user_agent=config["USER_AGENT"]
)
def fetch_posts_about_keyword(subreddit_name, keywords, limit=500000):
    subreddit = reddit.subreddit(subreddit_name)
    data = []
    
    today = datetime.utcnow()
    one_month_ago = today - timedelta(days=30)
    
    for post in subreddit.new(limit=limit):
        post_date = datetime.utcfromtimestamp(post.created)  
        
        if post_date < one_month_ago:
            continue
        
        matching_keywords = [keyword for keyword in keywords if keyword.lower() in post.title.lower()]

        for keyword in keywords:
            if keyword.lower() in post.title.lower():
                post.comments.replace_more(limit=0)
                comments = post.comments.list()

                for comment in comments:
                    data.append({
                        'post_title': post.title,
                        'post_score': post.score,
                        'post_id': post.id,
                        'post_num_comments': post.num_comments,
                        'post_created': post_date,  
                        'post_body': post.selftext,
                        'comment_id': comment.id,
                        'comment_body': comment.body,
                        'comment_score': comment.score,
                        'matched_keywords': ", ".join(matching_keywords)
                    })
                break 

    return data

subreddit_name = "stocks"  
keywords = ["Uber", "youtube", "meta", "apple", "nvidia", "microsoft", "amazon", "Saudi Arabian Oil Co", "intel", "tesla", "MARA Holdings"]

data = fetch_posts_about_keyword(subreddit_name, keywords, limit=5000)

df = pd.DataFrame(data)

df.to_csv("month_11_Reddit_comments.csv", index=False)
